# Advanced Scanning Techniques

This notebook demonstrates complex experimental patterns using rust-daq.

## Topics Covered

- 2D scans (grid patterns)
- Adaptive scans (data-driven positioning)
- Multi-detector scans
- Custom scan patterns

In [ ]:
from rust_daq import Motor, Detector, scan
from rust_daq.jupyter import quick_connect, LivePlot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

%matplotlib inline

In [ ]:
conn = quick_connect()
conn.__enter__()

## 2D Grid Scan

Scan two motors in a grid pattern to create a 2D map.

In [ ]:
# For this example, we'll use one motor and simulate 2D
# In practice, you'd use two different motors
motor_x = Motor("mock_stage")
detector = Detector("mock_power_meter")

def grid_scan(motor_x, detector, x_range, y_range, x_steps, y_steps):
    """
    Perform 2D grid scan.
    
    Note: This example uses one motor for demonstration.
    In practice, provide two Motor objects for true 2D scanning.
    """
    x_positions = np.linspace(x_range[0], x_range[1], x_steps)
    y_positions = np.linspace(y_range[0], y_range[1], y_steps)
    
    data = []
    total_points = x_steps * y_steps
    
    with tqdm(total=total_points, desc="2D Scan") as pbar:
        for y in y_positions:
            for x in x_positions:
                motor_x.position = x
                time.sleep(0.01)
                
                value = detector.read()
                data.append({'x': x, 'y': y, 'value': value})
                pbar.update(1)
    
    return pd.DataFrame(data)

# Execute 2D scan
grid_data = grid_scan(
    motor_x=motor_x,
    detector=detector,
    x_range=(0, 100),
    y_range=(0, 50),
    x_steps=20,
    y_steps=10
)

print(f"Collected {len(grid_data)} points")
grid_data.head()

In [ ]:
# Visualize 2D scan as heatmap
pivot_data = grid_data.pivot(index='y', columns='x', values='value')

plt.figure(figsize=(12, 6))
plt.imshow(pivot_data, aspect='auto', origin='lower', 
           extent=[0, 100, 0, 50], cmap='viridis')
plt.colorbar(label='Detector Reading')
plt.xlabel('X Position (mm)')
plt.ylabel('Y Position (mm)')
plt.title('2D Scan Heatmap')
plt.show()

## Adaptive Scan

Adjust scan parameters based on measured data.

In [ ]:
def adaptive_scan(motor, detector, initial_range, threshold, max_iterations=50):
    """
    Adaptively refine scan around regions of interest.
    
    Performs coarse scan, identifies peaks above threshold,
    then performs fine scan around those regions.
    """
    # Coarse scan
    print("Phase 1: Coarse scan...")
    coarse_positions = np.linspace(initial_range[0], initial_range[1], 20)
    coarse_data = []
    
    for pos in tqdm(coarse_positions, desc="Coarse"):
        motor.position = pos
        time.sleep(0.01)
        value = detector.read()
        coarse_data.append({'position': pos, 'value': value})
    
    coarse_df = pd.DataFrame(coarse_data)
    
    # Find peaks above threshold
    peaks = coarse_df[coarse_df['value'] > threshold]
    print(f"Found {len(peaks)} regions of interest above threshold {threshold}")
    
    # Fine scan around peaks
    fine_data = []
    if len(peaks) > 0:
        print("Phase 2: Fine scan around peaks...")
        for _, peak in peaks.iterrows():
            center = peak['position']
            # Scan ±5mm around peak with fine resolution
            fine_positions = np.linspace(center - 5, center + 5, 15)
            
            for pos in fine_positions:
                if initial_range[0] <= pos <= initial_range[1]:
                    motor.position = pos
                    time.sleep(0.01)
                    value = detector.read()
                    fine_data.append({'position': pos, 'value': value, 'scan_type': 'fine'})
    
    # Mark coarse data
    coarse_df['scan_type'] = 'coarse'
    
    # Combine results
    if fine_data:
        fine_df = pd.DataFrame(fine_data)
        return pd.concat([coarse_df, fine_df], ignore_index=True)
    else:
        return coarse_df

# Execute adaptive scan
adaptive_data = adaptive_scan(
    motor=motor_x,
    detector=detector,
    initial_range=(0, 100),
    threshold=5e-7  # Adjust based on your detector
)

print(f"Total data points: {len(adaptive_data)}")

In [ ]:
# Visualize adaptive scan
fig, ax = plt.subplots(figsize=(12, 6))

coarse = adaptive_data[adaptive_data['scan_type'] == 'coarse']
fine = adaptive_data[adaptive_data['scan_type'] == 'fine']

ax.plot(coarse['position'], coarse['value'], 'o-', label='Coarse scan', alpha=0.6)
if len(fine) > 0:
    ax.plot(fine['position'], fine['value'], 's-', label='Fine scan', alpha=0.8)

ax.set_xlabel('Position (mm)')
ax.set_ylabel('Detector Reading')
ax.set_title('Adaptive Scan Results')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## Custom Scan Patterns

Spiral, circular, or arbitrary patterns.

In [ ]:
def spiral_scan(motor, detector, max_radius, turns, points_per_turn):
    """
    Spiral scan pattern (using 1D motor for demonstration).
    """
    total_points = turns * points_per_turn
    theta = np.linspace(0, turns * 2 * np.pi, total_points)
    radius = np.linspace(0, max_radius, total_points)
    
    # Convert to linear positions for 1D motor
    positions = radius  # In real 2D, you'd have x = r*cos(theta), y = r*sin(theta)
    
    data = []
    for i, pos in enumerate(tqdm(positions, desc="Spiral")):
        motor.position = pos
        time.sleep(0.01)
        value = detector.read()
        data.append({
            'position': pos,
            'theta': theta[i],
            'radius': radius[i],
            'value': value
        })
    
    return pd.DataFrame(data)

spiral_data = spiral_scan(
    motor=motor_x,
    detector=detector,
    max_radius=100,
    turns=3,
    points_per_turn=20
)

In [ ]:
# Visualize spiral pattern
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Polar plot
ax1 = plt.subplot(121, projection='polar')
scatter = ax1.scatter(spiral_data['theta'], spiral_data['radius'], 
                     c=spiral_data['value'], cmap='plasma', s=50)
ax1.set_title('Spiral Scan (Polar)')
plt.colorbar(scatter, ax=ax1)

# Value vs radius
ax2 = plt.subplot(122)
ax2.plot(spiral_data['radius'], spiral_data['value'], 'o-', alpha=0.6)
ax2.set_xlabel('Radius (mm)')
ax2.set_ylabel('Detector Reading')
ax2.set_title('Value vs Radius')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Data Export and Analysis

In [ ]:
# Summary statistics
print("Grid Scan Statistics:")
print(grid_data['value'].describe())
print("\nAdaptive Scan Statistics:")
print(adaptive_data['value'].describe())
print("\nSpiral Scan Statistics:")
print(spiral_data['value'].describe())

In [ ]:
# Export to files
# grid_data.to_csv('grid_scan.csv', index=False)
# adaptive_data.to_csv('adaptive_scan.csv', index=False)
# spiral_data.to_csv('spiral_scan.csv', index=False)

# Or export to HDF5 for complex datasets
# grid_data.to_hdf('scans.h5', key='grid', mode='w')
# adaptive_data.to_hdf('scans.h5', key='adaptive')
# spiral_data.to_hdf('scans.h5', key='spiral')

In [ ]:
# Cleanup
conn.__exit__(None, None, None)